In [1]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.optimize import minimize

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$   
$
F(k,y(k;t)) = \frac{1}{2} \sum_{k=0}^{N} (x_{k}^{T}(t)Qx_{k}(t) + ru_{k}^{2})dt \\
mit \hspace{1cm}  u = kx_{k}  \hspace{1cm}  x_{k+1} = Ax_{k} + Bu_{k}  = (A+Bk)x_{k}
$

In [3]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [4]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([[0],[1/m],[0]])

C = np.array([[1,0,0]])
D = 0

In [102]:
global x0,N,nx, k0
f_dynamic = lambda x,u: A@x+B@u
k0 = np.array([[-2,-2,-2]])         # Anfangsschätzung
N = 50000                          # Anzahl von Zeitschritten N--> unendlich
nx = 3                             # Anzahl von Zuständen
x0 = np.array([[1],[0],[0]])       # Startzustand --> zu stabilisiern
dt = 0.001

Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B,C,D),dt, method='foh')  # Discretisierung damit dt --> gegen 0 geht  (= analytische Lösung)

def rollout(k,x):
    u     = k@x    
    dx    = Ad@x + Bd@u    
    #dx    =A@x +B*u    
    return dx , u

def cost(k):  
    k = np.reshape(k, (1, 3))   
    x_new, u   = rollout(k,x0)  

    cost_      = (x0.T@Q@x0 + R*u**2*0)/2*dt
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new, u = rollout(k,x_new)       
        
        cost_ += x_new.T@Q@x_new/2*dt
        cost_ += u**2*R/2*dt
   
    return cost_.flatten()

In [99]:
# Kontrolle der Kostenfunktion
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)

/tmp/ipykernel_29250/1038059239.py:2: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(cost, k0, method="nelder-mead", options={"disp": True})


Optimization terminated successfully.
         Current function value: 11.910897
         Iterations: 88
         Function evaluations: 159
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 11.9108965605448
             x: [-2.941e-01 -1.109e+00 -8.376e-01]
           nit: 88
          nfev: 159
 final_simplex: (array([[-2.941e-01, -1.109e+00, -8.376e-01],
                       [-2.941e-01, -1.109e+00, -8.377e-01],
                       [-2.941e-01, -1.109e+00, -8.376e-01],
                       [-2.941e-01, -1.109e+00, -8.376e-01]]), array([ 1.191e+01,  1.191e+01,  1.191e+01,  1.191e+01]))


In [100]:
# Alternative mit Scipy integrate
from scipy.integrate import quad
from scipy.linalg import expm

def integrand(t,k):
    k = np.reshape(k, (3, 1))
    x = expm((A + B@k.T)*t)@x0              # Lösung der Matrixespotenitlagleichung
    u = k.T@x                             # Eingang
    #print((x@Q@x + R*u**2)*0.5)
    return (x.T@Q@x + R*u**2)*0.5


cost3 = lambda k: quad(integrand, 0, np.inf, args=(k,))[0]

res3 = minimize(cost3, k0, method="nelder-mead", options={"disp": True})
print(res3)


/tmp/ipykernel_29250/1595469226.py:15: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res3 = minimize(cost3, k0, method="nelder-mead", options={"disp": True})


Optimization terminated successfully.
         Current function value: 11.917210
         Iterations: 84
         Function evaluations: 155
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 11.917209681931078
             x: [-3.092e-01 -1.134e+00 -9.129e-01]
           nit: 84
          nfev: 155
 final_simplex: (array([[-3.092e-01, -1.134e+00, -9.129e-01],
                       [-3.093e-01, -1.134e+00, -9.128e-01],
                       [-3.093e-01, -1.134e+00, -9.128e-01],
                       [-3.093e-01, -1.134e+00, -9.130e-01]]), array([ 1.192e+01,  1.192e+01,  1.192e+01,  1.192e+01]))


In [104]:
# Genenüberstellung
print('Kosten diskret ', cost(k0))
print('Kosten kontinuirlich', cost3(k0))

print('Zeitsikret endlich', res.x)
print('scipy integrade', res3.x)

Kosten diskret  [14.37269358]
Kosten kontinuirlich 14.375000000000004
Zeitsikret endlich [-0.29409068 -1.10878815 -0.83761751]
scipy integrade [-0.30923162 -1.13364782 -0.91285598]


## b) Gradient des Kostenfunktionals


In [112]:
def dcost(k):    
    gradient = np.zeros((nx,1),dtype=float)
    gradient += x0@k@x0*R*dt
    xnew,u   = rollout(k,x0)
    
    for i in range(0,N):                         
        #gradient += (xnew.T@Q).T  
        gradient  += (Ad+Bd@k).T@xnew *dt  
        gradient  += xnew@k@xnew*R *dt

        xnew,u  = rollout(k,xnew)
    
    return gradient

In [106]:
from scipy.optimize import approx_fprime

def dcost3(k):
    # Nummerisch Ableiten 
    return approx_fprime(k, cost3, 1e-8)

In [113]:

print(dcost(k0).T)
print(dcost3([-2,-2,-2]))

[[ 5.29416773e-03 -1.30070119e+01  4.00150065e+00]]
[-2.59999988  0.17499939  0.41249955]


## c) Gradientenverfahren nach [Gra19, Abschnitt 3.3.2]
Die Suchrichtung ergibt sich zu: $ \hspace{0.5cm} s^{k}= -\nabla f(x^{k}) $
## d) Schrittweitensteuerung mittels Backtracking Verfahren
mit dem Abbruchkriterium (Amerijo-Bedingung) der Schrittweitenadaption
![bedingung](./img/wolfe_bedingung.png)   
das Verfahren:   
![amerigo](./img/amerigo.png)



In [49]:
def backtracking(x,f,s,rho=0.5,c1=1e-7):

    alpha = 1   

    while f(x +alpha*s) >= f(x) + c1*alpha*s@s or np.isnan(f(x+alpha*s)):
        alpha *= rho   # Reduzierung der Schrittweite     
        if alpha < 1e-3:
            print('Error Schrittweite unterschritten')
            break

    return alpha

In [38]:
def gradienten_step(x,f,df,fix=False):
        
    # Gradientenrichtung --> ohne Regularisierung
    s = -df(x)               
    
    # Backtracking    
    alpha = backtracking(x,f,s)
    # Fixed Step size
    if fix == True:
        alpha = 0.1    
    
    print('Step size ', alpha)
    return x + alpha*s 


In [51]:
def linesearch(x0,f,df,Nmax=100,tol_f =1e-5, fix_stepsize=False ):
        
    x_alt = x0.copy()

    for k in range(Nmax):
              
        x = gradienten_step(x_alt,f,df,fix_stepsize)        
        # Abbruchkriterium
        if np.linalg.norm(df(x)) < tol_f:  
                print('Abbruch')          
                return x,k
        print(k, 'xopt= ',x, '                Kosten ', f(x))  
        x_alt = x.copy()
    return x,k
        

In [53]:
k0 = np.array([-2, -2, -2])
xopt, iter = linesearch(k0,cost3,dcost3, fix_stepsize = False)

print('xopt', xopt)
print('Iterationen', iter)

Step size  0.5
0 xopt=  [-0.70000006 -2.0874997  -2.20624977]                 Kosten  13.144728958977002


/tmp/ipykernel_29250/1595469226.py:10: RuntimeWarning: overflow encountered in matmul
  return (x.T@Q@x + R*u**2)*0.5
/tmp/ipykernel_29250/1595469226.py:10: RuntimeWarning: overflow encountered in square
  return (x.T@Q@x + R*u**2)*0.5
/home/stefan/.local/lib/python3.10/site-packages/scipy/linalg/_matfuncs.py:378: RuntimeWarning: overflow encountered in matmul
  eAw = eAw @ eAw
/tmp/ipykernel_29250/1595469226.py:7: RuntimeWarning: invalid value encountered in matmul
  x = expm((A + B@k.T)*t)@x0              # Lösung der Matrixespotenitlagleichung
/home/stefan/.local/lib/python3.10/site-packages/scipy/linalg/_matfuncs.py:378: RuntimeWarning: invalid value encountered in matmul
  eAw = eAw @ eAw
/tmp/ipykernel_29250/1595469226.py:13: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  cost3 = lambda k: quad(integrand, 0, np.inf, args=(k,))[0]
/tmp/ipykernel_29250/1595469226

Step size  0.125
1 xopt=  [-0.80987222 -1.68169688 -2.29722707]                 Kosten  12.212364608995703
Step size  0.25
2 xopt=  [-0.7018592  -1.51545413 -2.31972347]                 Kosten  12.168047958544353
Step size  0.125
3 xopt=  [-0.64066561 -1.6195436  -2.27683846]                 Kosten  12.137429989626282
Step size  0.125
4 xopt=  [-0.64283077 -1.54518583 -2.27376837]                 Kosten  12.120724389695768
Step size  0.25
5 xopt=  [-0.59791474 -1.5891369  -2.22004685]                 Kosten  12.120394442989841
Step size  0.125
6 xopt=  [-0.6109735  -1.5208473  -2.21311177]                 Kosten  12.106969199036051
Step size  0.125
7 xopt=  [-0.59611319 -1.54605811 -2.18396211]                 Kosten  12.100793152090992
Step size  0.25
8 xopt=  [-0.59681756 -1.49557121 -2.14954738]                 Kosten  12.09586053732093
Step size  0.125
9 xopt=  [-0.58046719 -1.53034697 -2.11811454]                 Kosten  12.088784844649522
Step size  0.25
10 xopt=  [-0.58594867 -1

## e) analytische Lösung
### algebraische Ricatti Gleichung

In [117]:
# Lösung der algebraischen Ricatti Gleichung
P = la.solve_continuous_are(A,B,Q,R)
K = -B.T@P/R
print('analytische Lösung',K)
Pd = la.solve_discrete_are(Ad,Bd,Q,R)
Kd = -Bd.T@Pd/R
print('zeitdiskrete Lösung',Kd)

analytische Lösung [[-0.30926715 -1.13366704 -0.91287093]]
zeitdiskrete Lösung [[-0.31306553 -1.13399859 -0.91338852]]


## f) Vergleich Sympy

In [115]:
res_Sympy = minimize(cost, k0, method="BFGS", options={"disp": True})
print('xopt = ', res_Sympy.x)

/tmp/ipykernel_29250/1775169149.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res_Sympy = minimize(cost, k0, method="BFGS", options={"disp": True})


Optimization terminated successfully.
         Current function value: 11.920033
         Iterations: 13
         Function evaluations: 68
         Gradient evaluations: 17
xopt =  [-0.30979274 -1.13339329 -0.91314938]


In [116]:
res_Sympy3 = minimize(cost3, k0, method="BFGS", options={"disp": True})
print('xopt = ', res_Sympy3.x)

/tmp/ipykernel_29250/1601048390.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res_Sympy3 = minimize(cost3, k0, method="BFGS", options={"disp": True})


Optimization terminated successfully.
         Current function value: 11.917210
         Iterations: 13
         Function evaluations: 68
         Gradient evaluations: 17
xopt =  [-0.3092671  -1.133667   -0.91287093]
